In [1]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
import os

genai.configure(
    api_key=os.environ['API_KEY'])
model_gemini = genai.GenerativeModel(
    model_name='gemini-pro')

# prompt = """
# You are an expert at solving word
# problems.

# Solve the following problem:

# I have three houses, each with three
# cats. Each cat owns 4 mittens, and a hat.
# Each mitten was knit from 7m of yarn,
# each hat from 4m. How much yarn was
# needed to make all the items?

# Think about it step by step, and show
# your work.
# """

# completion = model.generate_content(
#     prompt,
#     generation_config={
#         'temperature': 0,
#         'max_output_tokens': 800
#     }
# )

# print(completion.text)

e:\Research work\assessment checker research\my_protobuf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import docx2txt
import time
# import pandas
import docx
import re


# Function to apply gemini-3.5 model to a given text

In [15]:

# Define function to apply gemini-3.5 model to a given text
def generate_text(work, desc, rubrics):   
    description = desc
    rubrics=rubrics
    text = work
    instructor = """a language instructor and assess the work of students. I will provide you the assignment description/task given to students. 
        Then,  I will provide the student's work and the rubrics. 
        Your task will be to assign points and comments against each rubric first. 
        Then, Please highlight words, sentences, or phrases where the student made mistakes or violated rubrics.""" 
    prompt_aspect = f"""Act as {instructor} 
        Assignment description: 
        {description};    
        Rubrics:
        {rubrics}; 
        Student Work: 
        {text}"""

    try:
        completion = model_gemini.generate_content(
            prompt_aspect,
            generation_config={
                'temperature': 0.1,
                'max_output_tokens': 800
            }
        )
        score_gemini = completion.text
    except Exception as e:
        score_gemini = str(e) 
    result = score_gemini    
    # result = desc + rubrics+ text
    return result



# Functions to read files

In [16]:
# Define function to read text from a given Word file
def read_word_file(filename):

    # Open the Word file
    doc = docx.Document(filename)

    # Extract the text from the Word file
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)

    # Print the text from the Word file
    return "\n".join(text)

# Define function to read text from a given PDF file
def read_pdf_file(filename):
    with open(filename, 'rb') as f:
        reader = PyPDF2.PdfFileReader(f)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text



# Set essay id and Folder location

In [17]:
essay_id = 7
data_path = "asap-aes/"

# Read Rubrics and Descriptions

In [18]:
# desc = read_word_file(data_path+'Essay_Set_Descriptions/EssaySet'+str(essay_id)+'_Description.docx')
# rubrics = read_word_file(data_path+'Essay_Set_Descriptions/EssaySet'+str(essay_id)+'_Rubrics.docx')
prompt = read_word_file(data_path+'Essay_Set_Descriptions/Essay Set #'+str(essay_id)+'--ReadMeFirst.docx')
prompt= prompt.split('Rubric Guidelines', 1)
desc= prompt[0]
rubrics= prompt[1]
print(desc)

Essay Set #7
Prompt
Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.
Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.



In [19]:
print(rubrics)


A rating of 0-3 on the following four traits:
Ideas (points doubled)
Score 3: Tells a story with ideas that are clearly focused on the topic and are thoroughly developed with specific, relevant details.
Score 2: Tells a story with ideas that are somewhat focused on the topic and are developed with a mix of specific and/or general details.
Score 1: Tells a story with ideas that are minimally focused on the topic and developed with limited and/or general details.
Score 0: Ideas are not focused on the task and/or are undeveloped.
Organization
Score 3: Organization and connections between ideas and/or events are clear and logically sequenced. 
Score 2: Organization and connections between ideas and/or events are logically sequenced.
Score 1: Organization and connections between ideas and/or events are weak.
Score 0: No organization evident.
Style
Score 3: Command of language, including effective and compelling word choice and varied sentence structure, clearly supports the writer's purpos

In [20]:

# Create a list of unique rubric names
unique_rubrics = ["Score", "Point"]
# Create a new dictionary for each unique rubric name to store the assessment results
rubric_results = {rubric: [] for rubric in unique_rubrics}
# df.head(5)
keywords = ["overall statement", "Comments", "Mistakes"]
results = {rubric: [] for rubric in keywords}
feedback= []


# Call the process_files function

In [21]:
import pandas as pd
if(os.path.isfile(data_path+"results_gemini/EssaySet"+str(essay_id)+"_assessment_results_by_rubric.csv")):
    temp = pd.read_csv(data_path+"results_gemini/EssaySet"+str(essay_id)+"_assessment_results_by_rubric.csv")
else:
    temp=[]
print(len(temp))
# Load the data from a CSV file
df = pd.read_excel(data_path+"training_set_rel3.xlsx")
df = df[df['essay_set'] == essay_id]
# df = df[len(temp):]
df = df.dropna(axis=1, how='all')
print(len(df))
df.tail(2)


149
1569


,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4
12253,19562,7,I hate weddings. I love when people get marrie...,12.0,10.0,22.0,3.0,3.0,3.0,3.0,2.0,3.0,2.0,3.0
12254,19563,7,"A few weeks ago, we had a garage sale and a mo...",7.0,8.0,15.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0


# Loop to Iterate through the CSV

In [22]:
jump=2
csv_file_path = data_path+"results_gemini/EssaySet"+str(essay_id)+"_assessment_results_by_rubric.csv";
start = len(pd.read_csv(csv_file_path)) if os.path.exists(csv_file_path) else 0
print(start)
if os.path.exists(csv_file_path):
    pd.read_csv(csv_file_path).tail(2)

149


In [23]:

for i in range(0, 2, jump):
    data = df[i:i+jump] 
    # Iterate over the rows in the data
    for index, row in data.iterrows():

        # Get the text to assess
        text_to_assess = row["essay"]    
        print(text_to_assess)
        # Apply the OpenAI model to the text to get the assessment result
        assessment_result = generate_text(work= text_to_assess, desc=desc, rubrics=rubrics)
        feedback.append(assessment_result)    
        print(assessment_result)
        # Extract the rubric values from the assessment result text
        for rubric in unique_rubrics:
            pattern = re.compile(rf"(?i){rubric}\D+(\d+(\.\d+)?)")
            match = pattern.search(assessment_result)
            print(match)
            if match:
                rubric_results[rubric].append(match.group(1))
            else:
                rubric_results[rubric].append("")
        for i in range(len(keywords)):
            keyword = keywords[i]
            pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
            match = pattern.search(assessment_result)
            if match:
                results[keyword].append(match.group(1).strip())
            else:
                results[keyword].append("")
        time.sleep(10)            
        print(rubric_results)

    for rubric in unique_rubrics:
        data["domain1_"+rubric+"_gemini"] = rubric_results[rubric]
    for keyword in keywords:
        data[keyword+"_gemini"] = results[keyword]
    data['Response_gemini'] = feedback
    # Save the data with the rubric columns to a new CSV file
    # data.to_csv(data_path+"results_gemini/EssaySet"+str(essay_id)+"_assessment_results_by_rubric.csv", index = False, mode='a', header=False)
    data.to_csv(data_path+"results_gemini/EssaySet"+str(essay_id)+"_jhjkassessment_results_by_rubric.csv", index = False)
    rubric_results = {rubric: [] for rubric in unique_rubrics}
    results = {rubric: [] for rubric in keywords}
    feedback=[]

Patience is when your waiting .I was patience when in line waiting for lunch .I didn’t c ut any one to eat .I was standing and waiting for my turn .Patience ,some people don’t have it .Lots of people just cut or yell at you because they don’t have  any patience. Sometimes people will push you out of their way .They only do that because they don’t have patience at all. Patience is what people need .People need patience because lots o f feelings get hurt .Everyone should have patience.


**Ideas (6 points)**

* Score: 1
* Comments: The student's ideas are minimally focused on the topic and developed with limited and/or general details. The student provides a basic definition of patience but does not provide any specific examples or details to support their definition.

**Organization (2 points)**

* Score: 1
* Comments: The organization and connections between ideas and/or events are weak. The student's writing lacks a clear structure and logical flow of ideas.

**Style (2 points)**

* Score: 1
* Comments: The student's command of language is limited, including a lack of variety in word choice and sentences. The student's writing is simple and lacks descriptive language or figurative language.

**Conventions (1 point)**

* Score: 1
* Comments: The student's use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation is limited. The student makes several errors in grammar, usage, and punctuation.

**Total Score: 12 points**

**Hi

C:\Users\Nimra\AppData\Local\Temp\ipykernel_7772\1931516304.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["domain1_"+rubric+"_gemini"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_7772\1931516304.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["domain1_"+rubric+"_gemini"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_7772\1931516304.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using